In [1]:
val spark = SparkSession
  .builder()
  .appName("Bitcoin SCC")
  .config("spark.sql.broadcastTimeout", 36000)
  .config("spark.sql.cbo.enabled", true)
  //.config("spark.submit.deployMode", "cluster")
  .getOrCreate()

spark.sparkContext.setCheckpointDir("gs://bitcoin-hackathon/spark-checkpoint/")

val sc = spark.sparkContext

spark = org.apache.spark.sql.SparkSession@432fb17a
sc = org.apache.spark.SparkContext@6c7007c


org.apache.spark.SparkContext@6c7007c

In [ ]:
case class Address (Id: String)
case class Data(SignedBy_Address: String)
case class Spent(Data: Data, Target: String)

import spark.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.functions.{concat, concat_ws,explode,  typedLit}

val addressEncoder = implicitly[Encoder[Address]]
val spentEncoder = implicitly[Encoder[Spent]]

val addresses = spark.read.schema(addressEncoder.schema).json("gs://bitcoin-hackathon/epgm/address/output/part-00000.gz.txt")
val allAddresses = addresses.as[Address]

In [ ]:
val spent = spark.read.schema(spentEncoder.schema).json("gs://bitcoin-hackathon/epgm/SPENT/output/part-00000.gz.txt")
            .as[Spent]
allAddresses.map(a => (a.Id, spent.filter(s => s.Data.SignedBy_Address = a.Id)))

In [ ]:
import org.graphframes._

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val sqlContext = new org.apache.spark.sql.SQLContext(sc)

// 
val addresses = spark.read.json("gs://bitcoin-hackathon/epgm/address/output/part-00000.gz.txt","gs://bitcoin-hackathon/epgm/tx/output/part-00000.gz.txt")
        .select("Id","Meta.Label")
        .toDF



val spent = sqlContext.read.json("gs://bitcoin-hackathon/epgm/SPENT/output/")
spent.filter("Data.SignedBy_Address is null")
        .count


In [ ]:
val tran = sqlContext.read.json("gs://bitcoin-hackathon/epgm/tx/output")
                .filter("Id == '55d67c951dff3c5d5ccdeff39f29c9c8b3b2e239f492029c88b2c46e15d8f13f'")
tran.show(1, false)

In [ ]:
val tran = sqlContext.read.json("gs://bitcoin-hackathon/epgm/txout/output")
                .filter("Id == '55d67c951dff3c5d5ccdeff39f29c9c8b3b2e239f492029c88b2c46e15d8f13f-0'")
tran.show(1, false)

In [ ]:
val locked = sqlContext.read.json("gs://bitcoin-hackathon/epgm/LOCKED_AT/output")
                .filter("Source == '55d67c951dff3c5d5ccdeff39f29c9c8b3b2e239f492029c88b2c46e15d8f13f-0'")
locked.show(1, false)

In [ ]:
val vertices = spark.read
        .json(
            "gs://bitcoin-hackathon/epgm/address/output/",
            "gs://bitcoin-hackathon/epgm/block/output/",
            "gs://bitcoin-hackathon/epgm/tx/output/",
            "gs://bitcoin-hackathon/epgm/txout/output/")
        .select("Id","Meta.Label")
        .toDF("id","label")

val edges = spark.read
        .json(
            "gs://bitcoin-hackathon/epgm/LOCKED_AT/output/",
            "gs://bitcoin-hackathon/epgm/MINED_IN/output/",
            "gs://bitcoin-hackathon/epgm/PREVIOUS/output/",
            "gs://bitcoin-hackathon/epgm/PRODUCED/output/",
            "gs://bitcoin-hackathon/epgm/SPENT/output/")
        .select("Source", "Target", "Meta.Label")
        .toDF("src","dst","relationship")

// Create a GraphFrame
import org.graphframes.GraphFrame
val bitcoinGraph = GraphFrame(v, e)

bitcoinGraph.find("(a)<-[l]-(o); (o)-[s]->(t)")
    .filter("l.label == 'LOCKED_AT'")
    .filter("s.label == 'SPENT'")
    .select("l.Target","s.Data.SignedBy_Address","s.Target")
    .toDF("a1","a2","t")
    .show(10, false)

In [ ]:


val sqlContext = new org.apache.spark.sql.SQLContext(sc)
// import graphframes package
import org.graphframes._

// Create a Vertex DataFrame with unique ID column "id"
val v = sqlContext.createDataFrame(List(
    ("a1", "Address"),
    ("a2", "Address"),
    ("a3", "Address"),
    ("a4", "Address"),
    ("a5", "Address"),
    ("a6", "Address"),
    ("a7", "Address"),
    ("a8", "Address"),
    ("t1", "Tran"),
    ("t2", "Tran"),
    ("t3", "Tran"),
    ("t4", "Tran"),
    ("t5", "Tran")
)).toDF("id", "label")
// Create an Edge DataFrame with "src" and "dst" columns
val e = sqlContext.createDataFrame(List(
  ("a1", "t1", "spent"),
  ("a2", "t1", "spent"),
  ("a3", "t2", "spent"),
  ("a1", "t2", "spent"),
  ("a4", "t2", "spent"),
  ("a5", "t3", "spent"),
  ("a7", "t3", "spent"),
  ("a6", "t4", "spent"),
  ("a7", "t5", "spent"),
  ("a8", "t5", "spent")
)).toDF("src", "dst", "label")
// Create a GraphFrame
import org.graphframes.GraphFrame
val g = GraphFrame(v, e)

val result = g.connectedComponents.run()
result.filter("label == 'Address'").select("id", "component").orderBy("component").show()

In [ ]:
spark

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Dataset, Row, DataFrame, Column, DataFrameWriter, SaveMode}

val sqlContext = new org.apache.spark.sql.SQLContext(sc)
val add_tran_edges = sqlContext.read.csv("gs://bitcoin-hackathon/edges/address_to_transactions/*.gz")

val allAddresses = add_tran_edges
        .select("_c0")
        .distinct
        .withColumn("label",typedLit("Address"))
        .toDF("id","label")

val allTransactions = add_tran_edges
        .select("_c1")
        .distinct
        .withColumn("label",typedLit("Transactions"))
        .toDF("id","label")

val g_nodes = allAddresses.unionAll(allTransactions)
val g_edges = add_tran_edges.toDF("src","dst")

// Create a GraphFrame
import org.graphframes.GraphFrame
val g_bitcoin_tran_graph = GraphFrame(g_nodes, g_edges)

g_bitcoin_tran_graph.vertices.count
g_bitcoin_tran_graph.edges.count

val b_result = g_bitcoin_tran_graph.connectedComponents.run()
b_result.select("id", "label", "component")
    .orderBy("component")
    .write
    .option("compression","gzip")
    .format("csv") 
    .mode(SaveMode.Overwrite)
    .save("gs://bitcoin-hackathon/edges/address_cluster/")

sqlContext = org.apache.spark.sql.SQLContext@2be29f1f
add_tran_edges = [_c0: string, _c1: string]
allAddresses = [id: string, label: string]
allTransactions = [id: string, label: string]
g_nodes = [id: string, label: string]
g_edges = [src: string, dst: string]
g_bitcoin_tran_graph = GraphFrame(v:[id: string, label: string], e:[src: string, dst: string])


b_...


GraphFrame(v:[id: string, label: string], e:[src: string, dst: string])

In [11]:
var clusters = sqlContext.read.csv("gs://bitcoin-hackathon/edges/address_cluster/")

clusters = [_c0: string, _c1: string ... 1 more field]


639204230

In [12]:
g_nodes.count

639204230

In [13]:
g_edges.count

741853556

In [14]:
clusters.groupBy("_c2").count().orderBy(desc("count")).show
clusters.select("_c0").distinct.count

+-----+--------+
|  _c2|   count|
+-----+--------+
|   61|17838429|
|  102|16669982|
|    0| 6408688|
|  290| 2708379|
|   19| 2167701|
|51044| 2022072|
|  665| 1854454|
|  316| 1763323|
|   54| 1508485|
| 1947| 1217122|
|  431| 1033466|
|  386|  963054|
| 8465|  945670|
| 1342|  944787|
|   62|  928585|
|  486|  893157|
| 5936|  883223|
| 9912|  873785|
| 1199|  857866|
| 1226|  847407|
+-----+--------+
only showing top 20 rows



639204230